# Execute the trained model

In [2]:
import tensorflow as tf
import os
import re
import datetime
import numpy as np

/home/thomas/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/thomas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
model_folder = 'output'

In [4]:
from text_cnn import TextCNN

In [5]:
SAMPLE_LENGTH = 200

In [6]:
import gzip, pickle as pkl
with gzip.GzipFile("data/abridged_index2word.pkl.gz", "rb") as f:
    abridged_index2word = pkl.load(f)
with gzip.GzipFile("data/abridged_word2index.pkl.gz", "rb") as f:
    abridged_word2index = pkl.load(f)

In [7]:
cnn = TextCNN(
            sequence_length=SAMPLE_LENGTH,
            num_classes=3,
            vocab_size=len(abridged_index2word),
            embedding_size=300,
            filter_sizes=[3,4,5],
            num_filters=128)

In [8]:
sess = tf.InteractiveSession()

In [9]:
gensim_weights = [x for x in tf.global_variables() if "embedding" in str(x)]
other_weights = [x for x in tf.global_variables() if "embedding" not in str(x)]

In [10]:
saver = tf.train.Saver(other_weights)
saver_embedding = tf.train.Saver(gensim_weights)

In [11]:
saver_embedding.restore(sess, model_folder + "/gensim_weights")

INFO:tensorflow:Restoring parameters from output/gensim_weights


In [12]:
saver.restore(sess, "runs/checkpoints/model-100")

INFO:tensorflow:Restoring parameters from runs/checkpoints/model-100


In [13]:
import pickle as pkl, gzip, pandas as pd
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|\'')

In [14]:
def preprocess(text):
    tokens = tokenizer.tokenize(text)
    preprocessed  = []
    for token in tokens:
        if len(token) > 2 and token == token[0].upper() + token[1:].lower():
            token = "_PROPERNAME_"
        preprocessed.append(token.lower())
    return preprocessed

In [15]:
def preprocess_to_token_id(text):
    tokens = preprocess(text)
    ids = []
    for t in tokens:
        if t in abridged_word2index:
            idx = abridged_word2index[t]
        else: # _UNKNOWN_
            idx = 0
        ids.append(idx)
    return ids

In [16]:
# Letter by Anne Brontë
data= preprocess_to_token_id('''Sir,
      Ill health must plead my excuse for this long delay in acknowledging your flattering communication; but, believe me, I am not the less gratified at the pleasure you have derived from my own and my relatives' works, especially from the opinions they express. I have seen so little of controversial Theology that I was not aware the doctrine of Universal Salvation had so able and ardent an advocate as yourself; but I have cherished it from my very childhood - with a trembling hope at first, and afterwards with a firm and glad conviction of its truth. I drew it secretly from my own heart and from the word of God before I knew that any other held it. And since then it has ever been a source of true delight to me to find the same views either timidly suggested or boldly advocated by benevolent and thoughtful minds; and I now believe there are many more believers than professors in that consoling creed. Why good men should be so averse to admit it, I know not; - into their own hearts at least, however they might object to its promulgation among the bulk of mankind. But perhaps the world is not ripe for it yet. I have frequently thought that since it has pleased God to leave it in darkness so long respecting this particular truth, and often to use such doubtful language as to admit of such a general misconception thereupon, he must have some good reason for it. We see how liable men are to yield to the temptations of the passing hour; how little the dread of future punishment - how still less the promise of future reward can avail to make them forbear and wait; and if so many thousands rush into destruction with (as they suppose) the prospect of Eternal Death before their eyes, - what might not the consequence be, if that prospect were changed for one of a limited season of punishment, far distant and unseen, - however protracted and terrible it might be? 
      I thankfully cherish this belief; I honour those who hold it; and I would that all men had the same view of man's hopes and God's unbounded goodness as he had given to us, if it might be had with safety. But does not that if require some consideration? should we not remember the weak brother and the infatuated slave of satan, and beware of revealing these truths too hastily to those as yet unable to receive them? But in these suggestions I am perhaps condemning myself, for in my late novel, 'The Tenant of Wildfell Hall', I have given as many hints in support of the doctrine as I could venture to introduce into a work of that description. They are however mere suggestions, and as such I trust you will receive them, believing that I am well aware how much may be said in favour of boldly disseminating God's truth and leaving that to work its way. Only let our zeal be tempered with discretion, and while we labour, let us humbly look to God who is able and certain to bring his great work to perfection in his own good time and manner. 
      Accept my best wished in behalf of yourself and your important undertakings, and believe me to remain with sincere esteem

Yours truly 
        Acton Bell.
''')

In [17]:
# Letter by Charlotte Bronte
data= preprocess_to_token_id('''all I know – is that I cannot – that I will not resign myself to the total loss of my master’s friendship – I would rather undergo the greatest bodily pains that have my heart constantly lacerated by searing regrets. If my master withdraws his friendship from me entirely I shall be absolutely without hope – if he gives me a little friendship – a very little – I shall be content – happy, I would have a motive for living – for working.       Monsieur, the poor do not need a great deal to live on – they ask only the crumbs of bread which fall from the rich men’s table – but if they are refused these crumbs - they die of hunger -  No more do I need a great deal of affection from those I love – I would not know what to do with a whole and complete friendship – I am not accustomed to it – but you showed a little interest in me in days gone by when I was your pupil in Brussels – and I cling to the preservation of this little interest – I cling to it as I would cling on to life.

''')

In [18]:
len(data)

189

In [19]:
def execute_cnn(text_preprocessed_gensim):
    input_data = []
    if len(text_preprocessed_gensim) < SAMPLE_LENGTH:
        input_data.append([])
        for i in range(10):
            input_data[0].extend(text_preprocessed_gensim)
            if len(input_data[0]) > SAMPLE_LENGTH:
                input_data[0] = input_data[0][:SAMPLE_LENGTH]
                break
    else:
        for i in range(0, len(text_preprocessed_gensim) - SAMPLE_LENGTH - 1, 100):
            input_data.append(text_preprocessed_gensim[i:i+SAMPLE_LENGTH])
    
        
    result = sess.run(cnn.scores, {cnn.input_x: input_data, cnn.dropout_keep_prob: 1.0})
    
    proba_result = []
    for result_segment in result:
        probas = [np.exp(x) for x in result_segment]
        probas /= sum(probas)
        proba_result.append(probas)
    
    return np.asarray(proba_result)

In [20]:
d = execute_cnn(data)
d

array([[0.43884012, 0.35928553, 0.20187436]], dtype=float32)